# Pre-Process global _daily_ SST using `hot_to_blOb` to extract binary features

## Steps:
1. Compute Normalised Detrended Anomaly (cf. `hot_to_blOb.py::compute_normalised_anomaly()`)
2. Identify Extreme Values (i.e. above 95th percentile)

N.B.: Exploits parallelised `Dask` operations with optimised chunking using `flox` \
N.N.B.: This example using 40 years of Daily outputs at 0.25° resolution takes ~4 minutes on 128 cores

In [1]:
import xarray as xr
import dask
import intake
from getpass import getuser
from pathlib import Path

import spot_the_blOb.hot_to_blOb as hot
import spot_the_blOb.helper as hpc

In [2]:
# Start Dask Cluster
client = hpc.StartLocalCluster(n_workers=64, n_threads=2)

Memory per Worker: 7.87 GB
Hostname is  l40247
Forward Port = l40247:8787
Dashboard Link: localhost:8787/status


In [3]:
# Import 40 years of Daily EERIE ICON data

cat = intake.open_catalog("https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/eerie.yaml")
expid = 'eerie-control-1950'
version = 'v20231106'
model = 'icon-esm-er'
gridspec = 'gr025'

dat = cat['dkrz.disk.model-output'][model][expid][version]['ocean'][gridspec]

In [ ]:
# Load the data directly into optimal chunks

da_predictor = dat['2d_daily_mean'](chunks={}).to_dask().to.isel(depth=0).drop_vars('depth') # Test da
time_chunk = hot.rechunk_for_cohorts(da_predictor).chunks[0]

sst = dat['2d_daily_mean'](chunks={'time':time_chunk}).to_dask().to.isel(depth=0).drop_vars('depth')

In [ ]:
# Process Data using `hot_to_blOb` helper functions:

extreme_events_ds = hot.preprocess_data(sst, std_normalise=False, threshold_percentile=95)
extreme_events_ds

<xarray.Dataset> Size: 130GB
Dimensions:         (lat: 721, lon: 1440, time: 13879)
Coordinates:
  * lat             (lat) float64 6kB -90.0 -89.75 -89.5 ... 89.5 89.75 90.0
  * lon             (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time            (time) datetime64[ns] 111kB 2002-01-01T23:59:00 ... 2039-...
Data variables:
    dat_detrend     (time, lat, lon) float64 115GB dask.array<chunksize=(25, 721, 1440), meta=np.ndarray>
    mask            (lat, lon) bool 1MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    extreme_events  (time, lat, lon) bool 14GB dask.array<chunksize=(25, 721, 1440), meta=np.ndarray>
Attributes:
    description:           Standardised & Detrended Data
    preprocessing_steps:   ['Removed trend & seasonal cycle', 'Normalise by 3...
    threshold_percentile:  95

In [ ]:
# Save data to `zarr` for more efficient parallel I/O

file_name = Path('/scratch') / getuser()[0] / getuser() / 'mhws' / 'extreme_events_binary.zarr'
extreme_events_ds.to_zarr(file_name, mode='w')